In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
train = pd.read_csv('/content/Train_zsoft.csv')
test = pd.read_csv('/content/Test_zsoft.csv')

print(train.shape)
print(test.shape)

(30129, 3)
(7533, 2)


In [ ]:
train.head(5)

,ID,text,label
0,17428,TRANSPORT SYTEX / OEDA,Experience
1,15362,Appliquer la résine Avancé,Skills
2,37153,2020 - 2021 ¢ Formation en ligne sur le,Education
3,10240,Score: 820 (B2),Skills
4,29713,- Anglais :courant - Créole :courant,Skills


In [ ]:
# train['count'] = train['text'].apply(lambda x: len(x.split()))
# train.head()

In [ ]:
train['text'].str.len().max()
# train.head(10)

161

In [ ]:
train['encoded_text'] = train['label'].astype('category').cat.codes

train.head(5)


,ID,text,label,encoded_text
0,17428,TRANSPORT SYTEX / OEDA,Experience,1
1,15362,Appliquer la résine Avancé,Skills,2
2,37153,2020 - 2021 ¢ Formation en ligne sur le,Education,0
3,10240,Score: 820 (B2),Skills,2
4,29713,- Anglais :courant - Créole :courant,Skills,2


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import os
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Progress Bar
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
from transformers import logging as hf_logging
hf_logging.set_verbosity_error() # Hidding Huggingface Warnings
warnings.filterwarnings("ignore")

In [ ]:
stop_words = set(stopwords.words('french'))
porter = PorterStemmer()

# Remove all puncutation
def clean_responses(text):
    ## Remove all puncutation
    # string.punctuation = string.punctuation.replace('#','')
    # for punctuation in string.punctuation:
    #     text = text.replace(punctuation, '')
    
    ### Lowercase
    # text = text.lower()
    
    ### Removing stop words
    filtered_words = []
    for word in text.split():
        if word not in stop_words:
            filtered_words.append(word)
            
    ### Stemming words
    stemmed = [porter.stem(word) for word in filtered_words]
                
    return ' '.join(stemmed)

In [ ]:
train['text']=train['text'].apply(clean_responses)
# train_=train_.str.replace('\d+', '')

In [ ]:
train['text'].str.len().max()

161

In [ ]:
train.head()

,ID,text,label,encoded_text
0,17428,TRANSPORT SYTEX / OEDA,Experience,1
1,15362,Appliquer la résine Avancé,Skills,2
2,37153,2020 - 2021 ¢ Formation en ligne sur le,Education,0
3,10240,Score: 820 (B2),Skills,2
4,29713,- Anglais :courant - Créole :courant,Skills,2


# birt

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:

token = tokenizer.encode_plus(
    train['text'].iloc[0], 
    max_length=161, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
token.input_ids

<tf.Tensor: shape=(1, 161), dtype=int32, numpy=
array([[  101,   157,  9664, 12412, 23329, 10460,   156,  3663, 12880,
         3190,   120,   152, 10069,  1592,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
X_input_ids = np.zeros((len(train), 161))
X_attn_masks = np.zeros((len(train), 161))

In [ ]:

def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=161, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(train, X_input_ids, X_attn_masks, tokenizer)


30129it [00:34, 877.41it/s]


In [ ]:
labels = np.zeros((len(train), 3))
labels.shape

(30129, 3)

In [ ]:
labels[np.arange(len(train)), train['encoded_text'].values] = 1 # one-hot encoded target tensor


In [ ]:
labels


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data
     

<TakeDataset element_spec=(TensorSpec(shape=(161,), dtype=tf.float64, name=None), TensorSpec(shape=(161,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(DatasetMapFunction) # converting to required format for tensorflow dataset 


In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(161,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(161,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor


In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 161), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 161), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.float64, name=None))>

In [ ]:
p = 0.8
train_size = int((len(train)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [ ]:
train_size


1506

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
from transformers import TFBertModel


In [ ]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights


In [ ]:
input_ids = tf.keras.layers.Input(shape=(161,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(161,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
# X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(bert_embeds)
# X = tf.keras.layers.GlobalMaxPool1D()(X)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

model_ = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
model_.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 161)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 161)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 161,                                         

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model_.compile(optimizer=optim, loss=loss_func, metrics=[acc])


In [ ]:
hist = model_.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=4
)

Epoch 1/4
1506/1506 [==============================] - 854s 553ms/step - loss: 0.6515 - accuracy: 0.7290 - val_loss: 0.4789 - val_accuracy: 0.8165
Epoch 2/4
1506/1506 [==============================] - 831s 551ms/step - loss: 0.4845 - accuracy: 0.8078 - val_loss: 0.3657 - val_accuracy: 0.8639
Epoch 3/4
1506/1506 [==============================] - 830s 551ms/step - loss: 0.4011 - accuracy: 0.8456 - val_loss: 0.3245 - val_accuracy: 0.8806
Epoch 4/4
   9/1506 [..............................] - ETA: 12:27 - loss: 0.3275 - accuracy: 0.9028

KeyboardInterrupt: ignored

In [ ]:
model_.save('cv_model__')

In [ ]:
model_ = tf.keras.models.load_model('cv_model__')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=161, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=[0,1,2]):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
test['text']=test['text'].apply(clean_responses)
# test['text'].head()

In [ ]:
XX=[]
for i in range(7533):
  x__test= prepare_data(test['text'][i], tokenizer)
  XX.append(x__test)

In [ ]:
p=[]
for i in range(7533):
  pred=make_prediction(model_, processed_data=XX[i])
  p.append(pred)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
1/1 [==============================] - 0s 122ms/step


In [ ]:
p

[1,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 0,


In [ ]:
sub = pd.read_csv("SampleSubmission_zsoft.csv")
sub.head()

,ID,label
0,31507,Education
1,12238,Education
2,37232,Education
3,22248,Education
4,4589,Education


In [ ]:
sub['label'] = p
sub['label'] = sub["label"].replace([0,1,2],['Education','Experience','Skills'])
sub.set_index('ID',inplace = True)
sub.head(20)

,label
ID,
31507,Experience
12238,Education
37232,Education
22248,Skills
4589,Education
24893,Education
14299,Skills
6108,Experience
3942,Education


In [ ]:
sub.head(20)

,label
ID,
31507,Experience
12238,Education
37232,Education
22248,Skills
4589,Education
24893,Education
14299,Skills
6108,Experience
3942,Education


In [ ]:
sub.to_csv('submission1_zsoft_17.csv')